# CIFAR-10 Linear Model

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [2]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [2]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.7.0
Nom du GPU : /device:GPU:0
Metal device set to: Apple M1


2021-12-27 23:41:43.057863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-27 23:41:43.057951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Importation du dataset

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [5]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [6]:
LOG_DIR = os.path.join("logs")

## Fixer les seeds

In [7]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# Modèle linéaire

In [8]:
EPOCHS = 500
SHUFFLE = True
BATCH_SIZE = 256

In [9]:
class Perceptron(Model):
    def __init__(self, activation_function, kernel_initializer, l2_val=0, l1_val=0):
        super(Perceptron, self).__init__()
        self._flatten_layer = Flatten()
        if l1_val > 0 and l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1_l2(l1_val, l2_val),
                                      bias_regularizer=l1_l2(l1_val, l2_val)
                                      )
        elif l1_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1(l1_val),
                                      bias_regularizer=l1(l1_val)
                                      )
        elif l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l2(l2_val),
                                      bias_regularizer=l2(l2_val)
                                      )
        else:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer
                                      )
        self._output_layer = Dense(NUM_CLASSES, activation=softmax)
                
    
    def call(self, inputs):
        x = self._flatten_layer(inputs)
        x = self._dense_layer(x)
        return self._output_layer(x)

### Main Linear Model

Hyperparameters

In [ ]:
learning_rate = 0.01
momentum = 0.0
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

training

In [ ]:
perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

print(log_name)
perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

### learning rate analysis Models

hyperparameters

In [12]:
learning_rate = [0.05,0.2,0.001]
momentum = 0.0
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

training

In [ ]:
for lr in learning_rate:
    perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
    perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=lr,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

    log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

    print(log_name)
    perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

2021-12-27 23:42:23.175366: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-27 23:42:23.175408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


logs/linear/linear_model_ep_500_bs_256_opt_SGD_lr_0.05_mo_0.0__ki_GlorotUniform_af_tanh_pft


2021-12-27 23:42:23.587556: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-27 23:42:23.719394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/500
193/196 [============================>.] - ETA: 0s - loss: 3.4226 - categorical_accuracy: 0.2859

2021-12-27 23:42:26.985005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

196/196 [==============================] - 4s 21ms/step - loss: 259.4275 - categorical_accuracy: 0.0990 - val_loss: 246.6462 - val_categorical_accuracy: 0.1000
Epoch 2/500
196/196 [==============================] - 4s 20ms/step - loss: 265.6273 - categorical_accuracy: 0.1013 - val_loss: 337.1910 - val_categorical_accuracy: 0.1000
Epoch 3/500
196/196 [==============================] - 4s 21ms/step - loss: 250.0977 - categorical_accuracy: 0.1054 - val_loss: 240.4312 - val_categorical_accuracy: 0.1000
Epoch 4/500
196/196 [==============================] - 4s 21ms/step - loss: 211.0315 - categorical_accuracy: 0.1190 - val_loss: 166.1388 - val_categorical_accuracy: 0.1497
Epoch 5/500
196/196 [==============================] - 4s 21ms/step - loss: 189.3229 - categorical_accuracy: 0.1275 - val_loss: 137.1945 - val_categorical_accuracy: 0.1657
Epoch 6/500
196/196 [==============================] - 4s 21ms/step - loss: 174.2457 - categorical_accuracy: 0.1325 - val_loss: 126.7258 - val_categoric

2021-12-28 00:48:22.169016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


193/196 [============================>.] - ETA: 0s - loss: 2.2111 - categorical_accuracy: 0.2000

2021-12-28 00:48:25.534490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 5s 23ms/step - loss: 2.2102 - categorical_accuracy: 0.2006 - val_loss: 2.1217 - val_categorical_accuracy: 0.2605
Epoch 2/500
196/196 [==============================] - 4s 19ms/step - loss: 2.0742 - categorical_accuracy: 0.2759 - val_loss: 2.0340 - val_categorical_accuracy: 0.2932
Epoch 3/500
196/196 [==============================] - 4s 19ms/step - loss: 2.0079 - categorical_accuracy: 0.3022 - val_loss: 1.9853 - val_categorical_accuracy: 0.3174
Epoch 4/500
196/196 [==============================] - 4s 21ms/step - loss: 1.9662 - categorical_accuracy: 0.3194 - val_loss: 1.9491 - val_categorical_accuracy: 0.3298
Epoch 5/500
196/196 [==============================] - 4s 20ms/step - loss: 1.9362 - categorical_accuracy: 0.3320 - val_loss: 1.9231 - val_categorical_accuracy: 0.3396
Epoch 6/500
196/196 [==============================] - 4s 19ms/step - loss: 1.9133 - categorical_accuracy: 0.3411 - val_loss: 1.9035 - val_categorical_accuracy: 0.3468
Epoc

Resultat:


### Momentum analysis Models

In [14]:
learning_rate = 0.01
momentum = [0.1, 0.5, 0.9]
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

training

In [ ]:
for mt in momentum:
    perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
    perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=mt),
                   metrics=categorical_accuracy)

    log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{mt}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

    print(log_name)
    perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

logs/linear/linear_model_ep_500_bs_256_opt_SGD_lr_0.01_mo_0.1__ki_GlorotUniform_af_tanh_pft
Epoch 1/500
  8/196 [>.............................] - ETA: 3s - loss: 2.3344 - categorical_accuracy: 0.1367

2021-12-28 01:50:37.033112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


195/196 [============================>.] - ETA: 0s - loss: 1.9861 - categorical_accuracy: 0.2907

2021-12-28 01:50:40.256020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 4s 20ms/step - loss: 1.9860 - categorical_accuracy: 0.2908 - val_loss: 1.9577 - val_categorical_accuracy: 0.2814
Epoch 2/500
196/196 [==============================] - 3s 18ms/step - loss: 1.8483 - categorical_accuracy: 0.3499 - val_loss: 1.8874 - val_categorical_accuracy: 0.3241
Epoch 3/500
196/196 [==============================] - 3s 18ms/step - loss: 1.8002 - categorical_accuracy: 0.3702 - val_loss: 1.8234 - val_categorical_accuracy: 0.3608
Epoch 4/500
196/196 [==============================] - 3s 17ms/step - loss: 1.7801 - categorical_accuracy: 0.3784 - val_loss: 1.8434 - val_categorical_accuracy: 0.3362
Epoch 5/500
196/196 [==============================] - 4s 18ms/step - loss: 1.7599 - categorical_accuracy: 0.3867 - val_loss: 1.7669 - val_categorical_accuracy: 0.3801
Epoch 6/500
196/196 [==============================] - 4s 19ms/step - loss: 1.7445 - categorical_accuracy: 0.3936 - val_loss: 1.8046 - val_categorical_accuracy: 0.3735
Epoc

...

196/196 [==============================] - 5s 23ms/step - loss: 1.9415 - categorical_accuracy: 0.3082 - val_loss: 1.9117 - val_categorical_accuracy: 0.3018
Epoch 2/500
196/196 [==============================] - 4s 21ms/step - loss: 1.8111 - categorical_accuracy: 0.3668 - val_loss: 1.8109 - val_categorical_accuracy: 0.3609
Epoch 3/500
196/196 [==============================] - 4s 21ms/step - loss: 1.7661 - categorical_accuracy: 0.3836 - val_loss: 1.7693 - val_categorical_accuracy: 0.3780
Epoch 4/500
196/196 [==============================] - 4s 21ms/step - loss: 1.7490 - categorical_accuracy: 0.3887 - val_loss: 1.7563 - val_categorical_accuracy: 0.3849
Epoch 5/500
196/196 [==============================] - 4s 21ms/step - loss: 1.7231 - categorical_accuracy: 0.4007 - val_loss: 1.7572 - val_categorical_accuracy: 0.3777
Epoch 6/500
196/196 [==============================] - 4s 20ms/step - loss: 1.7070 - categorical_accuracy: 0.4051 - val_loss: 1.7685 - val_categorical_accuracy: 0.3907
Epoc

2021-12-28 02:57:59.963330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


195/196 [============================>.] - ETA: 0s - loss: 1.9420 - categorical_accuracy: 0.3212

2021-12-28 02:58:03.676455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 5s 24ms/step - loss: 1.9419 - categorical_accuracy: 0.3211 - val_loss: 1.9142 - val_categorical_accuracy: 0.3308
Epoch 2/500
196/196 [==============================] - 4s 21ms/step - loss: 1.7903 - categorical_accuracy: 0.3720 - val_loss: 1.8125 - val_categorical_accuracy: 0.3653
Epoch 3/500
196/196 [==============================] - 4s 22ms/step - loss: 1.7024 - categorical_accuracy: 0.4010 - val_loss: 1.7210 - val_categorical_accuracy: 0.3968
Epoch 4/500
196/196 [==============================] - 4s 22ms/step - loss: 1.6581 - categorical_accuracy: 0.4172 - val_loss: 1.6831 - val_categorical_accuracy: 0.4096
Epoch 5/500
196/196 [==============================] - 4s 21ms/step - loss: 1.6242 - categorical_accuracy: 0.4300 - val_loss: 1.6244 - val_categorical_accuracy: 0.4260
Epoch 6/500
196/196 [==============================] - 4s 21ms/step - loss: 1.5918 - categorical_accuracy: 0.4424 - val_loss: 1.5854 - val_categorical_accuracy: 0.4409
Epoc

### Kernel Initializer analysis Models

hyperparameters

In [16]:
learning_rate = 0.01
momentum = 0.0
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = he_normal
fa = tanh
version = '_pft'

training

In [17]:
perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

print(log_name)
perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

logs/linear/linear_model_ep_500_bs_256_opt_SGD_lr_0.01_mo_0.0__ki_HeNormal_af_tanh_pft
Epoch 1/500
  4/196 [..............................] - ETA: 3s - loss: 2.4688 - categorical_accuracy: 0.1094 

2021-12-28 03:44:59.656309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


193/196 [============================>.] - ETA: 0s - loss: 1.9887 - categorical_accuracy: 0.2884

2021-12-28 03:45:03.869256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 5s 25ms/step - loss: 1.9881 - categorical_accuracy: 0.2888 - val_loss: 1.9413 - val_categorical_accuracy: 0.2890
Epoch 2/500
196/196 [==============================] - 4s 18ms/step - loss: 1.8477 - categorical_accuracy: 0.3482 - val_loss: 1.8944 - val_categorical_accuracy: 0.3165
Epoch 3/500
196/196 [==============================] - 4s 19ms/step - loss: 1.7938 - categorical_accuracy: 0.3718 - val_loss: 1.8104 - val_categorical_accuracy: 0.3640
Epoch 4/500
196/196 [==============================] - 4s 18ms/step - loss: 1.7702 - categorical_accuracy: 0.3788 - val_loss: 1.8224 - val_categorical_accuracy: 0.3442
Epoch 5/500
196/196 [==============================] - 4s 19ms/step - loss: 1.7474 - categorical_accuracy: 0.3888 - val_loss: 1.7486 - val_categorical_accuracy: 0.3876
Epoch 6/500
196/196 [==============================] - 4s 19ms/step - loss: 1.7301 - categorical_accuracy: 0.3968 - val_loss: 1.7931 - val_categorical_accuracy: 0.3766
Epoc

### Activation function analysis Models

Hyperparameters

In [18]:
learning_rate = 0.01
momentum = 0.0
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = relu
version = '_pft'

training

In [19]:
perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

print(log_name)
perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

logs/linear/linear_model_ep_500_bs_256_opt_SGD_lr_0.01_mo_0.0__ki_GlorotUniform_af_relu_pft
Epoch 1/500
  4/196 [..............................] - ETA: 3s - loss: 2.3371 - categorical_accuracy: 0.1260 

2021-12-28 04:18:39.472202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


194/196 [============================>.] - ETA: 0s - loss: 1.9968 - categorical_accuracy: 0.2898

2021-12-28 04:18:42.992405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 5s 24ms/step - loss: 1.9964 - categorical_accuracy: 0.2899 - val_loss: 1.9133 - val_categorical_accuracy: 0.3079
Epoch 2/500
196/196 [==============================] - 4s 21ms/step - loss: 1.8422 - categorical_accuracy: 0.3577 - val_loss: 1.8362 - val_categorical_accuracy: 0.3449
Epoch 3/500
196/196 [==============================] - 4s 22ms/step - loss: 1.7842 - categorical_accuracy: 0.3797 - val_loss: 1.7819 - val_categorical_accuracy: 0.3810
Epoch 4/500
196/196 [==============================] - 4s 22ms/step - loss: 1.7506 - categorical_accuracy: 0.3927 - val_loss: 1.7767 - val_categorical_accuracy: 0.3582
Epoch 5/500
196/196 [==============================] - 4s 22ms/step - loss: 1.7211 - categorical_accuracy: 0.4033 - val_loss: 1.7122 - val_categorical_accuracy: 0.4110
Epoch 6/500
196/196 [==============================] - 4s 22ms/step - loss: 1.6967 - categorical_accuracy: 0.4141 - val_loss: 1.7231 - val_categorical_accuracy: 0.4009
Epoc

### Batch size analysis Models

hyperparameters

In [10]:
BATCH_SIZE = [512,1024,128,32]
learning_rate = 0.01
momentum = 0.0
l2_val = 0.00
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

training

In [11]:
for bs in BATCH_SIZE:
    perceptron = Perceptron(fa, ki, l1_val=l1_val, l2_val=l2_val)
    perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

    log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{bs}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

    print(log_name)
    perceptron.fit(x_train, y_train, batch_size=bs, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

2021-12-28 11:20:48.084321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-28 11:20:48.084968: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.01_mo_0.0__ki_GlorotUniform_af_tanh_pft


2021-12-28 11:20:48.593428: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-28 11:20:48.740672: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/500
98/98 [==============================] - ETA: 0s - loss: 2.0607 - categorical_accuracy: 0.2569

2021-12-28 11:20:51.163946: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

1563/1563 [==============================] - 18s 11ms/step - loss: 0.0451 - categorical_accuracy: 0.9950 - val_loss: 2.4932 - val_categorical_accuracy: 0.5110
Epoch 285/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0449 - categorical_accuracy: 0.9955 - val_loss: 2.4859 - val_categorical_accuracy: 0.5144
Epoch 286/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0456 - categorical_accuracy: 0.9951 - val_loss: 2.4425 - val_categorical_accuracy: 0.5215
Epoch 287/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0450 - categorical_accuracy: 0.9952 - val_loss: 2.4585 - val_categorical_accuracy: 0.5158
Epoch 288/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0435 - categorical_accuracy: 0.9957 - val_loss: 2.4180 - val_categorical_accuracy: 0.5260
Epoch 289/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0421 - categorical_accuracy: 0.9960 - val_loss: 2.4172 - val_categ

1563/1563 [==============================] - 17s 11ms/step - loss: 0.0259 - categorical_accuracy: 0.9985 - val_loss: 2.5488 - val_categorical_accuracy: 0.5229
Epoch 332/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0260 - categorical_accuracy: 0.9987 - val_loss: 2.5617 - val_categorical_accuracy: 0.5228
Epoch 333/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0258 - categorical_accuracy: 0.9985 - val_loss: 2.5498 - val_categorical_accuracy: 0.5247
Epoch 334/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0253 - categorical_accuracy: 0.9989 - val_loss: 2.5517 - val_categorical_accuracy: 0.5245
Epoch 335/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0253 - categorical_accuracy: 0.9986 - val_loss: 2.5592 - val_categorical_accuracy: 0.5286
Epoch 336/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0257 - categorical_accuracy: 0.9986 - val_loss: 2.5450 - val_categ

1563/1563 [==============================] - 18s 11ms/step - loss: 0.0171 - categorical_accuracy: 0.9996 - val_loss: 2.6385 - val_categorical_accuracy: 0.5260
Epoch 379/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0170 - categorical_accuracy: 0.9995 - val_loss: 2.6437 - val_categorical_accuracy: 0.5247
Epoch 380/500
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0170 - categorical_accuracy: 0.9996 - val_loss: 2.6457 - val_categorical_accuracy: 0.5259
Epoch 381/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0168 - categorical_accuracy: 0.9994 - val_loss: 2.6406 - val_categorical_accuracy: 0.5276
Epoch 382/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0166 - categorical_accuracy: 0.9997 - val_loss: 2.6530 - val_categorical_accuracy: 0.5276
Epoch 383/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0164 - categorical_accuracy: 0.9996 - val_loss: 2.6432 - val_categ

1563/1563 [==============================] - 18s 11ms/step - loss: 0.0126 - categorical_accuracy: 0.9998 - val_loss: 2.7169 - val_categorical_accuracy: 0.5263
Epoch 426/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0123 - categorical_accuracy: 0.9998 - val_loss: 2.7328 - val_categorical_accuracy: 0.5261
Epoch 427/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0123 - categorical_accuracy: 0.9999 - val_loss: 2.7252 - val_categorical_accuracy: 0.5238
Epoch 428/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0123 - categorical_accuracy: 0.9999 - val_loss: 2.7325 - val_categorical_accuracy: 0.5237
Epoch 429/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0123 - categorical_accuracy: 0.9998 - val_loss: 2.7281 - val_categorical_accuracy: 0.5245
Epoch 430/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0121 - categorical_accuracy: 0.9999 - val_loss: 2.7470 - val_categ

1563/1563 [==============================] - 18s 11ms/step - loss: 0.0097 - categorical_accuracy: 0.9999 - val_loss: 2.8369 - val_categorical_accuracy: 0.5198
Epoch 473/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0097 - categorical_accuracy: 0.9999 - val_loss: 2.7855 - val_categorical_accuracy: 0.5263
Epoch 474/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0097 - categorical_accuracy: 0.9999 - val_loss: 2.7934 - val_categorical_accuracy: 0.5260
Epoch 475/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0094 - categorical_accuracy: 1.0000 - val_loss: 2.7902 - val_categorical_accuracy: 0.5249
Epoch 476/500
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0096 - categorical_accuracy: 1.0000 - val_loss: 2.7976 - val_categorical_accuracy: 0.5241
Epoch 477/500
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0095 - categorical_accuracy: 0.9999 - val_loss: 2.7878 - val_categ

Resultat: